# OCR Prototypes

Rerun the first code cell to reload the files if new files are added

In [2]:
# Install the ipynb package
#%pip install ipynb

# Base code for all notebooks in the same folder
from ipynb.fs.full.common import load_files, reload_packages, setup # type: ignore

setup()

## Reload Packages()

Rerun the cell below this one to reload common packages in repo_packages or workspace_packages if they change

In [29]:

reload_packages()

import cv2 # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pytesseract # type: ignore
import io

from cls_img_tools import ImageTools
from cls_string_helpers import StringHelpers

from PIL import Image, ImageOps, ImageFilter # type: ignore

import os

def no_op():
    pass

def resize_image_pillow(image_path: str, new_width: int=1200):
    with Image.open(image_path) as img:
        original_width, original_height = img.size
        aspect_ratio = original_height / original_width
        new_height = int(aspect_ratio * new_width)

        resized_img = img.resize((new_width, new_height), Image.LANCZOS)
        return resized_img, new_height

def resize_image_opencv(image_path: str, new_width: int = 1200) -> np.ndarray:
    # Read the image
    img = cv2.imread(image_path) #, cv2.IMREAD_UNCHANGED)
    #display_image_opencv(img)

    #image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # Normalize the 16-bit image to 8-bit
    #image_8bit = cv2.convertScaleAbs(img, alpha=(255.0/65535.0))
    #display_image_opencv(image_8bit)

    # Get the original dimensions
    original_height, original_width = img.shape[:2]

    # Calculate the new dimensions
    aspect_ratio = original_height / original_width
    new_height = int(aspect_ratio * new_width)

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
    #display_image_opencv(resized_img)

    return resized_img, new_height

def crop_image_opencv(image: np.ndarray, x: int, y: int, width: int, height: int) -> np.ndarray:
    # Crop the image using array slicing
    cropped_img = image[y:y+height, x:x+width]
    return cropped_img

def enhance_image(img, scale=2) -> Image.Image:
    img = img.convert('L')  # Convert the image to grayscale

    # Apply binary thresholding
    img = img.point(lambda x: 0 if x < 225 else 255, '1')

    # Resize the image
    img = img.resize((img.width * scale, img.height * scale), Image.LANCZOS)

    # Sharpen the image
    img = img.filter(ImageFilter.SHARPEN)

    return img

def adjust_gamma(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                    for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def enhance_image_opencv(img, scale=2) -> np.ndarray:
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Step 1: Enhance contrast with CLAHE
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    #image = clahe.apply(image)

    # Step 2: Apply gamma correction
    image = adjust_gamma(image, gamma=1.5)

    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return binary

def convert_to_black_and_white_opencv(image: np.ndarray, threshold: int = 128) -> np.ndarray:
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    return binary

def convert_non_white_to_black_opencv(image, threshold=200):
    """
    Convert non-white pixels to black in the given image using OpenCV.
    """
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a mask for pixels that are above the threshold
    mask = gray > threshold

    # Create an output image with the same shape as the input image, initialized to black
    output = np.zeros_like(image)

    # Set the pixels that are above the threshold in the original image to white in the output image
    output[mask] = [255, 255, 255]

    return output

def is_white(pixel, threshold=225):
    r, g, b = pixel
    return r > threshold and g > threshold and b > threshold

def convert_non_white_to_black(image, threshold=225):
    # Ensure the image is in RGB mode
    img = image.convert('RGB')

    # Create a new image with the same size
    new_img = Image.new('RGB', img.size)

    # Process each pixel
    for x in range(img.width):
        for y in range(img.height):
            pixel = img.getpixel((x, y))
            if is_white(pixel):
                new_img.putpixel((x, y), (255, 255, 255))  # Keep white pixels as white
            else:
                new_img.putpixel((x, y), (0, 0, 0))  # Convert non-white pixels to black

    return new_img

def display_image(img, title="Image", rotate=0):
    # Rotate the image 90 degrees clockwise for display
    img = img.rotate(rotate, expand=True)

    # Display the image using matplotlib
    plt.imshow(img)
    plt.title(title)

    plt.axis('off')  # Hide the axis
    plt.show()

def display_image_opencv(img, title="Image"):
    # Display the image using matplotlib
    plt.imshow(img)
    plt.title(title)

    plt.axis('off')  # Hide the axis
    plt.show()


Reloading packages


## Prototypes

In [56]:
import re
import easyocr

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

files = []
files = load_files("../png_samples/team_helps_scores")

for img_file in files:
    img, new_height = resize_image_opencv(img_file)

    print(f"Processing {img_file} with new height: {new_height}")

    player_results = []

    offset_height = 600
    players_img = crop_image_opencv(img, 300, offset_height, 440, new_height - offset_height) # Only player tags
    players_img = convert_non_white_to_black_opencv(players_img, 225)
    #display_image_opencv(players_img, title="Players Image")

    results = reader.readtext(players_img, mag_ratio=2.0)
    for box, text, confidence in results:
        if box[0][0] < 50:
            player_results.append((box, text, confidence))
        #print(f"Player: {player[1]} - Box y {player[0][0][1]}")

    player_helps = []

    helps_img = crop_image_opencv(img, 740, offset_height, 150, new_height - offset_height)
    helps_img = convert_non_white_to_black_opencv(helps_img, 245)
    #display_image_opencv(helps_img, title="Helps Image")
    results = reader.readtext(helps_img, mag_ratio=2.0, allowlist="0123456789")
    for box, text, confidence in results:
        player_helps.append((box, text, confidence))
        #print(f"Helps: {help[1]} - Box y {help[0][0][1]}")

    player_stars = []

    stars_img = crop_image_opencv(img, 890, offset_height, 250, new_height - offset_height)
    stars_img = convert_non_white_to_black_opencv(stars_img, 245)
    #display_image_opencv(stars_img, title="Stars Image")
    results = reader.readtext(stars_img, mag_ratio=2.0, allowlist="0123456789,")
    for box, text, confidence in results:
        player_stars.append((box, text, confidence))
        #print(f"Stars: {text} - Box y {box[0][1]}")

    print(f"Length of player results: {len(player_results)}")
    print(f"Length of player helps: {len(player_helps)}")
    print(f"Length of player stars: {len(results)}")

    # Match numeric values with text values based on y-coordinate
    matches = []
    unmatched = []
    for player_text_box, player_text, player_text_confidence in player_results:
        player_y = player_text_box[0][1]  # y-coordinate of the top-left corner of the text box
        helps_stars = (None, None)
        for help_text_box, help_text, help_text_confidence in player_helps:
            help_y = help_text_box[0][1]  # y-coordinate of the top-left corner of the numeric box
            if -40 <= (player_y - help_y) <= 25:
                helps_stars = (help_text, None)
                break;

        # if we didn't find a Helps match it's because it was 0 which EasyOCR is not recognizing
        if helps_stars[0] is None:
            helps_stars = (0, None)

        for star_text_box, star_text, star_text_confidence in results:
            star_y = star_text_box[0][1]
            if -40 <= (player_y - star_y) <= 25:
                helps_stars = (helps_stars[0], star_text)
                break

        if helps_stars[1] is None:
            unmatched.append((player_text, helps_stars))
        else:
            matches.append((player_text, helps_stars))

for match in matches:
    print(f"Player: {match[0]} - Helps: {match[1][0]} - Stars: {match[1][1]}")

for unmatch in unmatched:
    print(f"Unmatched: {unmatch}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for files in /Users/tedbouskill/Repos/MyGitHub/wordscape-score-scraper/prototypes/../png_samples/team_helps_scores
Processing /Users/tedbouskill/Repos/MyGitHub/wordscape-score-scraper/prototypes/../png_samples/team_helps_scores/2025-01-05-02.png with new height: 2596
Length of player results: 9
Length of player helps: 6
Length of player stars: 9
Player: cariann - Helps: 26 - Stars: 1,165,394
Player: Laura - Helps: 14 - Stars: 1,162,651
Player: gardener - Helps: 44 - Stars: 1,115,938
Player: Punches616 - Helps: 0 - Stars: 1,095,956
Player: chibong - Helps: 0 - Stars: 1,071,979
Player: Quinn - Helps: 17 - Stars: 1,019,510
Player: Stranger - Helps: 40 - Stars: 1,000,265
Player: jon - Helps: 0 - Stars: 936,822
Player: Silev - Helps: 64 - Stars: 884,631


### Prototype for Import Scores

In [5]:
from json import load
import easyocr

from re import S
from tokenize import String
from sympy import O # type: ignore

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

min_score_x = 1200

files = []
files = load_files("../png_samples/weekend_scores")

for img_file in files:
    if "reduced" in img_file or "IMG" in img_file:
        continue

    print(f"Processing file: {img_file}")

    img, new_height = resize_image_opencv(img_file)
    #display_image_opencv(img, title="Original Image")

    #img = enhance_image_opencv(img)

    #header_img = img.crop((0, 0, 1200, 1050))
    #display_image(header_img, title="Header Image")

    state_img = crop_image_opencv(img, 450, 680, 300, 100)
    state_img = convert_non_white_to_black_opencv(state_img)
    state_txt = pytesseract.image_to_string(state_img)
    if state_txt is not None:
        state_txt = state_txt.strip()
        print(f"State: {state_txt}")
    #display_image_opencv(state_img, title="State Image")

    # If tournament fininshed, the rank image is present
    if state_txt == "FINISHED":
        rank_img = crop_image_opencv(img, 100, 540, 200, 110)
        rank_img = convert_non_white_to_black_opencv(rank_img)
        rank_txt = pytesseract.image_to_string(rank_img).strip()
        print(f"Rank: {rank_txt}")
        #display_image_opencv(rank_img, title="Rank Image")

    #ranks_img = crop_image_opencv(img, 40, 1000, 110, new_height - 1000)
    #ranks_img = enhance_image_opencv(ranks_img)
    #results = reader.readtext(ranks_img)
    #for box, text, confidence in results:
    #    print(f"Box: {box}, Text: {text}, Confidence: {confidence}")
    #display_image_opencv(ranks_img, title="Ranks Image")

    player_results = []
    score_results = []
    results = []

    players_img = crop_image_opencv(img, 300, 1050, 600, new_height - 1050) # Only player tags
    #players_img = crop_image_opencv(img, 300, 1050, 900, new_height - 1050) # Player and score tags
    players_img = convert_non_white_to_black_opencv(players_img, 225)
    display_image_opencv(players_img, title="Players Image")
    results = reader.readtext(players_img, mag_ratio=2.0)
    for box, text, confidence in results:
        print(f"Text: {text}, Box: {box}, Confidence: {confidence}")
        # If the y is less than 100, then it is the player name
        if box[0][0] < 50:
            player_results.append((box, text, confidence))

        if box[0][0] > 650: # and (StringHelpers.is_all_numeric(text) or "O" in text or "o" in text):
            if box[0][0] < min_score_x:
                min_score_x = box[0][0]
            score_results.append((box, text, confidence))

    scores_img = crop_image_opencv(img, 900, 1050, 300, new_height - 1050)
    scores_img = convert_non_white_to_black_opencv(scores_img, 222)
    scores_img = adjust_gamma(scores_img, gamma=5.1)
    display_image_opencv(scores_img, title="Scores Image")
    #results = reader.detect(scores_img, mag_ratio=4.0)
    #for bbox in results[0]:
    #    print(bbox)  # Print bounding boxes to check if zeros are detected

    results = reader.readtext(scores_img, detail=1, mag_ratio=4.0, allowlist="0123456789oO", blocklist=".,!@#$%^&*()")
    for box, text, confidence in results:
        print(f"Text: {text}, Box: {box}, Confidence: {confidence}")
        score_results.append((box, text, confidence))
    tesseract_results = pytesseract.image_to_string(scores_img, config='--psm 7 -c tessedit_char_whitelist=0123456789Oo')
    print(f"Tesseract: {tesseract_results}")

    print("Length of player results: ", len(player_results))
    print("Length of score results: ", len(score_results))
    print("Min score x: ", min_score_x)

     # Match numeric values with text values based on y-coordinate
    matches = []
    unmatched = []
    for text_box, text, text_confidence in player_results:
        text_y = text_box[0][1]  # y-coordinate of the top-left corner of the text box
        matched = False
        for num_box, num_text, num_confidence in score_results:
            num_y = num_box[0][1]  # y-coordinate of the top-left corner of the numeric box
            if 0 < abs(num_y - text_y) < 65:
                matches.append((text, num_text))
                matched = True
                break
        if not matched:
            unmatched.append((text, text_confidence))

    # Print matched results
    for text, num in matches:
        print(f"Text: {text}, Numeric: {num}")

    # Print unmatched results
    print("Unmatched:")
    for text, confidence in unmatched:
        print(f"Text: {text}, Confidence: {confidence}")

    #display_image_opencv(players_img, title="Players Image")

    print()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for files in /Users/tedbouskill/Repos/MyGitHub/wordscape-score-scraper/prototypes/../png_samples/weekend_scores


### Text Found Metrics using EasyOCR

In [23]:
import easyocr
import re

import cls_string_helpers as StringHelpers

from fuzzywuzzy import fuzz, process
from hmac import new

def get_min_max_values(boxes):
    if not boxes:
        return None, None, None, None

    min_height = min(min(box[0][1], box[1][1], box[2][1], box[3][1]) for box in boxes)
    min_width = min(min(box[0][0], box[1][0], box[2][0], box[3][0]) for box in boxes)
    max_height = max(max(box[0][1], box[1][1], box[2][1], box[3][1]) for box in boxes)
    max_width = max(max(box[0][0], box[1][0], box[2][0], box[3][0]) for box in boxes)

    return min_height, min_width, max_height, max_width

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

tt_ds = set()
ts_dims = set()
team_rank_dims = set()
teammates_dims = set()
first_dims = set()
second_dims = set()
third_dims = set()
scores_dims = set()

max_image_heights = []
unique_strings = set()
unique_uppercase_strings = set()

files = []
files = load_files("../png_samples/weekend_scores")

for img_file in files:
    temp_path, new_height = resize_image_pillow(img_file)
    max_image_heights.append(new_height)

    results = reader.readtext(temp_path)

    for box, text, confidence in results:
        box_tuple = tuple(map(tuple, box))  # Convert list of lists to tuple of tuples

        if StringHelpers.is_all_uppercase(text):
            if text == "TEAM TOURNAMENT":
                tt_ds.add(box_tuple)
                continue
            if text == "FINISHED":
                ts_dims.add(box_tuple)
                continue
            if text == "TEAMMATES":
                teammates_dims.add(box_tuple)
                continue

            similarity_ratio = fuzz.ratio(text, "TEAM TOURNAMENT")
            if similarity_ratio > 75:
                tt_ds.add(box_tuple)
                continue

            unique_uppercase_strings.add(text)

            continue

        if StringHelpers.is_all_numeric(text):
            nbr = StringHelpers.convert_to_integer(text)

            if nbr > 50 and box[0][0] > 600:
                scores_dims.add(box_tuple)
                continue

            if text == "1":
                first_dims.add(box_tuple)
                continue
            if text == "2":
                second_dims.add(box_tuple)
                continue
            if text == "3":
                third_dims.add(box_tuple)
                continue

            continue

        if text.startswith("#"):
            team_rank_dims.add(box_tuple)
            continue

        unique_strings.add(text)

        #print(f"\tBox: {box_tuple}, Text: {text}, Confidence: {confidence}")

    os.remove(temp_path)

max_image_height = max(max_image_heights)
print(f"MAX IMAGE HEIGHT {max_image_height}")

# Collect min/max values for each set of boxes
data = [
    ("TEAM TOURNAMENT", *get_min_max_values(tt_ds)),
    ("FINISHED", *get_min_max_values(ts_dims)),
    ("TEAMMATES", *get_min_max_values(teammates_dims)),
    ("FIRST", *get_min_max_values(first_dims)),
    ("SECOND", *get_min_max_values(second_dims)),
    ("THIRD", *get_min_max_values(third_dims)),
    ("SCORES", *get_min_max_values(scores_dims)),
    ("TEAM RANK", *get_min_max_values(team_rank_dims))
]

print(unique_uppercase_strings)
print(unique_strings)

# Print CSV formatted output
print("Label,min_height,min_width,max_height,max_width")
for row in data:
    print(",".join(map(str, row)))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


MAX IMAGE HEIGHT 2596
{'CONTINUE', 'RANKING', 'REWARD', 'RANK'}
{'prizes will be divided by 50.', 'lichi', 'Dewey', 'TEAM -OURNAENT', '25,000', 'mike', '~EAV TOURNAMEN-', 'zmewis', 'dinogirl', "Win prizes based on your team's stars! The team", 'keymony', '1d 4h', 'allenge yourselfl', 'chibong', 'xi Solitaire Trigeak', 'kay', 'JoCo', '1d4h', 'Not Participating', 'Stranger', 'butterfly', 'allongo vcursomi', 'Goose', 'spudly', 'ki Solitaire Tripeak', 'Mar', 'Siley', 'Murphy', 'tedbilly', 'cariann', 'Suriel', 'RNnCinci', "Win prizes based on your team'$ stars! The team", 'Jay'}
Label,min_height,min_width,max_height,max_width
TEAM TOURNAMENT,177,178,280,1025
FINISHED,690,481,750,722
TEAMMATES,843,668,1041,1080
FIRST,1081,81,1243,105
SECOND,1288,77,1456,109
THIRD,1500,77,1665,109
SCORES,629,925,2540,1114
TEAM RANK,542,144,633,270
